In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import math
import plotly.express as px
from sklearn.neighbors import LocalOutlierFactor

tqdm.pandas()

In [ ]:
## Run the following to compute the mappings for host locations
#!python3 -m scripts.mappings_host_location_all_cities

## Load Visualization Pipeline
#!python3 -m scripts.pipeline_all_cities_viz

## Load ML pipeline
#!python3 -m scripts.pipeline_ML

In [ ]:
df = pd.read_pickle("data/pickles/total_listings_viz.pkl")

In [ ]:
df.head()

In [ ]:
df.columns.tolist()

## Visualize neighbourhoods

In [ ]:
neigh_list = df["neighbourhood_cleansed"].tolist()
neigh_counter = {x:neigh_list.count(x) for x in neigh_list}
neigh_counter = dict(sorted(neigh_counter.items(), key=lambda item: item[1], reverse=True))

neigh_counter


## Testing

In [ ]:
df = pd.read_pickle("data/pickles/total_listings_exploration_handling.pkl")

In [ ]:
df.head()

In [ ]:
df["listing_city"].unique().tolist()

In [ ]:
fig = px.histogram(df.loc[df["price"]<1000], x = "price")
fig.show()

In [ ]:
df.loc[df["price"]>=400, "price_category"] = "high_high_price"
df.loc[df["price"]<400, "price_category"] = "normal_high_price"
df.loc[df["price"]<160, "price_category"] = "normal_medium_price"
df.loc[df["price"]<100, "price_category"] = "normal_low_price"
df.loc[df["price"]<60, "price_category"] = "low_low_price"



In [ ]:
fig = px.histogram(df, x = "price_category")
fig.show()

In [ ]:
df["host_response_rate"].unique()

In [ ]:
fig = px.scatter(df, x = "price", y = "host_response_rate")
fig.show()

In [ ]:
df_only_num = pd.DataFrame(df.dtypes, columns=["type"]).loc[(pd.DataFrame(df.dtypes, columns=["type"])["type"] == "float64") | (pd.DataFrame(df.dtypes, columns=["type"])["type"] == "int64")]

In [ ]:
df_only_num = df[df_only_num.index.tolist()]

In [ ]:
df.loc[df["price"]>5000, :]

In [ ]:
df.loc[df["price_category"]=="normal_price", :]

In [ ]:
# This shows that the coordinates distributions are multimodal distributions
# Nonetheless, I use the StandardScaler to scale the data
fig = px.ecdf(df, x = "x_coord")
fig.show()

In [ ]:
df.head()

In [ ]:
fig = px.histogram(df, x = "x_coord")
fig.show()

In [ ]:

fig = px.histogram(df, x = "y_coord")
fig.show()


In [ ]:
fig = px.histogram(df, x = "z_coord")
fig.show()

### Normalize numerical data distribution with `power_transform`

In [ ]:
fig = px.histogram(df.loc[df["number_of_reviews"]<100], x = "number_of_reviews")
fig.show()

In [ ]:
fig = px.histogram(df, x = "accommodates")
fig.show()


In [ ]:
numerical_positive = [
    "host_listings_count",
    "host_location",
    "accommodates",
    "number_of_reviews",
    "reviews_per_month",
    "amenities_benefits"
]

In [ ]:
df[numerical_positive].skew().sort_values(ascending=False)

In [ ]:
df.columns.tolist()

## Cross-city confrontation

In [ ]:
city_confr_price = pd.DataFrame(df.groupby(["df_city_location"]).median("price")["price"])
city_confr_price.reset_index(inplace=True)

fig = px.histogram(city_confr_price,
                   x = "df_city_location",
                   y = "price"
                   )
fig.update_xaxes(categoryorder="total descending")
fig.show()

In [ ]:
city_confr_review_scores_rating = pd.DataFrame(df.groupby(["df_city_location"]).median("review_score_rating")["review_scores_rating"])
city_confr_review_scores_rating.reset_index(inplace=True)

fig = px.histogram(city_confr_review_scores_rating,
                   x = "df_city_location",
                   y = "review_scores_rating"
                   )
fig.update_xaxes(categoryorder="total descending")
fig.show()


In [ ]:
city_confr_num_reviews = pd.DataFrame(df.groupby(["df_city_location"]).median("number_of_reviews")["number_of_reviews"])
city_confr_num_reviews.reset_index(inplace=True)

fig = px.histogram(city_confr_num_reviews,
                   x = "df_city_location",
                   y = "number_of_reviews"
                   )
fig.update_xaxes(categoryorder="total descending")
fig.show()

In [ ]:
city_confr_accommodates = pd.DataFrame(df.groupby(["df_city_location"]).median("accommodates")["accommodates"])
city_confr_accommodates.reset_index(inplace=True)

fig = px.histogram(city_confr_accommodates,
                   x = "df_city_location",
                   y = "accommodates"
                   )
fig.update_xaxes(categoryorder="total descending")
fig.show()


In [ ]:
city_confr_population = pd.DataFrame(df.groupby(["df_city_location"]).median("listing_city_pop")["listing_city_pop"])
city_confr_population.reset_index(inplace=True)

fig = px.histogram(city_confr_population,
                   x = "df_city_location",
                   y = "listing_city_pop"
                   )
fig.update_xaxes(categoryorder="total descending")
fig.show()


## Neighbourhoods

In [ ]:
fig = px.scatter(df,
                 x = "listing_city_pop",
                 y = "price",
                 log_x=True,
                 log_y=True
                 )
fig.show()